In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Set up directories and parameters
base_dir = 'Medicinal Leaf Dataset/Segmented Medicinal Leaf Images'
batch_size = 32
img_size = (299, 299)

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2  # 20% for validation
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
    subset='validation'
)

# Load Inception V3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Create the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(30, activation='softmax')  # Adjust for your number of classes
])

# Compile the model with the correct learning rate parameter
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10  # Adjust as necessary
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

# Save the model
model.save('leaf_identification_model.h5')

# Function to predict class of a new image
def predict_image(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions, axis=1)
    class_label = list(train_generator.class_indices.keys())[class_idx[0]]
    return class_label

# Example usage of prediction function
# predicted_class = predict_image('path_to_your_image.jpg')
# print(f'The predicted class is: {predicted_class}')


Found 1479 images belonging to 30 classes.
Found 356 images belonging to 30 classes.
Epoch 1/10


C:\Users\singh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 8/47 ━━━━━━━━━━━━━━━━━━━━ 2:53 4s/step - accuracy: 0.1005 - loss: 3.3863